# install moabb & braindecode

In [1]:
pip install moabb

     |████████████████████████████████| 122kB 25.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 27.4MB 118kB/s 
     |████████████████████████████████| 6.8MB 37.2MB/s 
     |████████████████████████████████| 7.0MB 41.8MB/s 
     |████████████████████████████████| 645kB 36.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install braindecode

     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 133kB 37.0MB/s 


In [3]:
pip install -U https://api.github.com/repos/shivar2/braindecode/zipball/master

     \ 225kB 14.2MB/s
  Created wheel for Braindecode: filename=Braindecode-0.5-cp37-none-any.whl size=67275 sha256=89a3db727223a10084f52d0cdbf9fb16f137f0407d36454212eae38313ffe58e
  Stored in directory: /tmp/pip-ephem-wheel-cache-zr2z11qo/wheels/e8/96/7e/9b71798fc04d312683ade7e3a639b36579b720176c6a54dc86
Successfully built Braindecode
  Found existing installation: Braindecode 0.5.1
    Uninstalling Braindecode-0.5.1:
      Successfully uninstalled Braindecode-0.5.1


# connect to drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os, sys
os.chdir('/content/drive/MyDrive/MotorImagery-Classification')

# preproccess

## set MNE config

In [6]:
import mne

mne.utils.get_config()

mne.set_config('MNE_DATASETS_BNCI_PATH', 'mne_data')


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


<ipython-input-6-c2a0ce2fb86f>:5: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BNCI_PATH"
  mne.set_config('MNE_DATASETS_BNCI_PATH', 'mne_data')


## download or load dataset

In [12]:
from braindecode.datasets.moabb import MOABBDataset
subject_id = 1
dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=[subject_id])

SHA256 hash of downloaded file: 7e731ee8b681d5da6ecb11ae1d4e64b1653c7f15aad5d6b7620b25ce53141e80
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: d4229267ec7624fa8bd3af5cbebac17f415f7c722de6cb676748f8cb3b717d97
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 15850d81b95fc88cc8b9589eb9b713d49fa071e28adaf32d675b3eaa30591d6e
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 81916dff2c12997974ba50ffc311da006ea66e525010d010765f0047e771c86a
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 7738

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
4

## preproccess

In [13]:
from braindecode.datautil.preprocess import (
    exponential_moving_standardize, preprocess, Preprocessor)

low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda x: x * 1e6),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data

preprocess(dataset, preprocessors)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 

## save sets

In [15]:
dataset.save(path='data-file/bnci-raw/' + str(subject_id))

Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/0-raw.fif
Closing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/0-raw.fif
[done]
Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/1-raw.fif
Closing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/1-raw.fif
[done]
Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/2-raw.fif
Closing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/2-raw.fif
[done]
Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/3-raw.fif
Closing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/3-raw.fif
[done]
Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/4-raw.fif
Closing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-raw/4-raw.fif
[done]
Writing /content/drive/My Drive/MotorImagery-Classification/data-file/bnci-